MultiRAT preprocessing
================
Joanes Grandjean

![rat art](../assets/img/rat_art.png)

# Purpose
In this section I submit jobs to my high performance computer to perform the preprocessing, confound_correction, and analysis steps as implemented in RABIES. 
This section relies on the fact that my environment allows for parallel job submission using the `qsub` command. 



In [1]:
# init variables
init_folder='/home/traaffneu/joagra/code/MultiRat'  # location of the codes
analysis_folder='/scratch/m/mchakrav/gjoanes/multirat/test'      # location of the bids folder and results
df_path='../assets/table/meta_data_20211208.tsv'    # meta-data table

scratch_folder='/scratch/m/mchakrav/gjoanes/multirat/test/scratch' # this is where preprocessed data will be temporarily stored
rabies_img='/project/m/mchakrav/gjoanes/rabies-0.3.5.simg' # location of the RABIES image

In [2]:
import os
import pandas as pd
import numpy as np

df = pd.read_csv(df_path, sep='\t')  # load the table

In [3]:
! ssh gjoanes@niagara.computecanada.ca -tt 'sh ~/detect_error.sh && exit' > ../assets/tmp/confound_redo_tmp.txt

If you are having trouble logging in, please see:
https://docs.computecanada.ca/wiki/SSH_Keys_Pilot 

Si vous rencontrez des difficultés pour vous authentifier, veuillez consulter: 
https://docs.computecanada.ca/wiki/SSH_Keys_Pilot/fr

Connection to niagara.computecanada.ca closed.


In [4]:
confound_redo = pd.read_csv('../assets/tmp/confound_redo_tmp.txt', header=None)

In [5]:
from subprocess import call


module_args = "module load gcc/8.3.0; module load openblas/0.3.7; module load fsl/6.0.4"  #the freesurfer and ants modules tricks as specific to my system due to some env contamination bugs 
fsl_load_args='module load fsl'
fsl_unload_args='module unload fsl'

template_args = "template=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Template.nii; mask=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Mask.nii; wm=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_WM_bin.nii.gz; csf=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_CSF_bin.nii.gz; atlas=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Brain_Atlases/SIGMA_Anatomical_Atlas/SIGMA_Anatomical_Brain_Atlas_rs.nii; roi=/template/roi/"

rabies_confound_args = "confound_correction /rabies_out /confound_out \
--read_datasink \
--highpass 0.01 \
--smoothing_filter 0.5 ${param}"

rabies_analysis_args = "analysis /confound_out /analysis_out \
--FC_matrix \
--seed_list /roi_folder/ACA_l.nii.gz /roi_folder/CPu_l.nii.gz /roi_folder/MOp_l.nii.gz /roi_folder/S1bf_l.nii.gz"

rabies_confound_aromas = "denoise='aromas'; param='--lowpass 0.1 --run_aroma --aroma_dim 10'"
rabies_confound_aromasr = "denoise='aromasr'; param='--lowpass 0.1 --run_aroma --aroma_dim 10 --FD_censoring'"
rabies_confound_aromal = "denoise='aromasl'; param='--lowpass 0.2 --run_aroma --aroma_dim 10'"
rabies_confound_GSRs ="denoise='GSRs'; param='--lowpass 0.1 --conf_list global_signal mot_6'"
rabies_confound_WMCSFs="denoise='WMCSFs'; param='--lowpass 0.1 --conf_list WM_signal CSF_signal mot_6'"

In [7]:
for index, row in confound_redo.iterrows():

    # get information from dataframe 
    log_name=row[0].replace('/scratch/m/mchakrav/gjoanes/multirat/test/export/log/confound//','')
    subject_ID=log_name.split('_')[0]
    session_ID=log_name.split('_')[1]
    confound=log_name.split('_')[2].replace('.log','')
    TR="TR="+str(df[(df['rat.sub']==int(subject_ID.replace('sub-0',''))) & (df['rat.ses']==int(session_ID.replace('ses-','')))]['func.TR'].values[0])

    if confound == 'aromas': rest1_args = rabies_confound_aromas
    elif confound == 'aromasr': rest1_args = rabies_confound_aromasr
    elif confound == 'aromal': rest1_args = rabies_confound_aromal
    elif confound == 'GSRs': rest1_args = rabies_confound_GSRs
    elif confound == 'WMCSFs': rest1_args = rabies_confound_WMCSFs

    # set the base folder structure and init folders
    origbids_args="orig_folder="+os.path.join(analysis_folder, "bids", subject_ID,session_ID)
    template_folder_args="template_folder="+os.path.join(analysis_folder, "template")
    folderbids_args="bids_folder="+os.path.join(scratch_folder, "bids", subject_ID+"_"+session_ID)
    folderprep_args="preprocess_folder="+os.path.join(scratch_folder, "preprocess", subject_ID+"_"+session_ID)
    rest3_args='current_confound='+os.path.join(scratch_folder,'confound')+'/${denoise}/'+subject_ID+'_'+session_ID
    rest4_args='current_analysis='+os.path.join(scratch_folder,'analysis')+'/${denoise}/'+subject_ID+'_'+session_ID
    rest5_args='rm -rf $current_confound; rm -rf $current_analysis; mkdir -p $current_confound; mkdir -p $current_analysis'

    # get error message if any error occurred during confound regression 
    rest_error1_args='now=`date`; ! grep -q "Error" ${current_confound}/rabies_confound_correction.log || echo $now" "'+analysis_folder+'/export/log/confound/' + subject_ID+'_'+session_ID+'_${denoise}.log >> '+analysis_folder+'export/log/error.log'

    # copy aromas confound outputs of interest to designated folders
    rest_cp1_args='cp ${current_confound}/rabies_confound_correction.log ' +analysis_folder+ '/export/log/confound/' + subject_ID+'_'+session_ID+'_${denoise}.log'
    rest_cp2_args='if [ "$denoise" == "aromas" ]; then cp -r ${current_confound}/confound_correction_wf_datasink/cleaned_timeseries/*/*.nii.gz '+analysis_folder+'/export/aromas_scan; ls ${current_confound}/confound_correction_wf_datasink/aroma_out | while read label; do mkdir -p '+analysis_folder+'/export/aromas_qa/${label}; cp -r ${current_confound}/confound_correction_wf_datasink/aroma_out/${label}/aroma_out/melodic.ica/report/* '+analysis_folder+'/export/aromas_qa/${label}; cp -r ${current_confound}/confound_correction_wf_datasink/aroma_out/${label}/aroma_out/classified_motion_ICs.txt '+analysis_folder+'/export/aromas_qa/${label}/classification.txt; done;fi'

    # get error message if any error occurred during analysis 
    rest_error2_args='now=`date`; ! grep -q "Error" ${current_analysis}/rabies_analysis.log || echo $now" "'+analysis_folder+'/export/log/analysis/' + subject_ID+'_'+session_ID+'_${denoise}.log >> '+analysis_folder+'export/log/error.log'


    # copy seed-based maps to designated folders
    rest_cp3_args='cp ${current_analysis}/rabies_analysis.log ' +analysis_folder+ '/export/log/analysis/' + subject_ID+'_'+session_ID+'_${denoise}.log'
    rest_cp4_args='seed_dir='+analysis_folder+'/export/seed/${denoise}; mkdir -p ${seed_dir}; cp -r ${current_analysis}/analysis_wf_datasink/seed_correlation_maps/*/*/* $seed_dir'
    rest_cp5_args='NA_dir='+analysis_folder+'/export/NA/${denoise}; mkdir -p ${NA_dir}; cp -r ${current_analysis}/analysis_wf_datasink/matrix_data_file/*/* $NA_dir'


    #report completed script to log file. Doesnt mean completed is error free
    complete_args='echo $now" "'+subject_ID+'_'+session_ID+' >> '+analysis_folder+'/export/log/complete.log'

    qsub_job_name=os.path.join('/project/4180000.19/multiRat', 'qsub','redo', 'redo_job_'+subject_ID+'_'+session_ID +'_'+confound+'.sh')

    with open (qsub_job_name, 'w') as rsh:
        rsh.writelines("#"+'!/bin/bash' +'\n')
        rsh.writelines("#"+'SBATCH --time=06:00:00' +'\n')
        rsh.writelines("#"+'SBATCH --nodes=1' +'\n')
        rsh.writelines("#"+'SBATCH --account=rrg-mchakrav-ab' +'\n')
        rsh.writelines("#"+'SBATCH --ntasks=40' +'\n')
        rsh.writelines(module_args +'\n')
        rsh.writelines(template_args +'\n')
        rsh.writelines(TR +'\n')
        rsh.writelines(origbids_args +'\n')
        rsh.writelines(template_folder_args +'\n')
        rsh.writelines(folderbids_args +'\n')
        rsh.writelines(folderprep_args +'\n')
        # add if statement here for resting state
        rsh.writelines(rest1_args +'\n')
        rsh.writelines(rest3_args +'\n')
        rsh.writelines(rest4_args +'\n')
        rsh.writelines(rest5_args +'\n')
        rsh.writelines('singularity run -B ${template_folder}:/template -B ${bids_folder}:/rabies_in:ro ')
        rsh.writelines('-B ${preprocess_folder}:/rabies_out -B ${current_confound}:/confound_out ')
        rsh.writelines(rabies_img+' ')
        rsh.writelines(rabies_confound_args +'\n')
        rsh.writelines(rest_error1_args +'\n')
        rsh.writelines(rest_cp1_args +'\n')
        rsh.writelines(rest_cp2_args +'\n')
        rsh.writelines('singularity run -B ${template_folder}/roi:/roi_folder -B ${template_folder}:/template -B ${bids_folder}:/rabies_in:ro ')
        rsh.writelines('-B ${preprocess_folder}:/rabies_out -B ${current_confound}:/confound_out -B ${current_analysis}:/analysis_out ')
        rsh.writelines(rabies_img+' ')
        rsh.writelines(rabies_analysis_args +'\n')
        rsh.writelines(rest_error2_args +'\n')
        rsh.writelines(rest_cp3_args +'\n')
        rsh.writelines(rest_cp4_args+'\n')
        rsh.writelines(rest_cp5_args+'\n')
        #rsh.writelines(clear_args +'\n')
        rsh.writelines(complete_args +'\n')


In [8]:
! cd /project/4180000.19/multiRat/qsub/redo
! scp -r redo_job*  gjoanes@niagara.computecanada.ca://scratch/m/mchakrav/gjoanes/multirat/test/qsub/redo

If you are having trouble logging in, please see:
https://docs.computecanada.ca/wiki/SSH_Keys_Pilot 

Si vous rencontrez des difficultés pour vous authentifier, veuillez consulter: 
https://docs.computecanada.ca/wiki/SSH_Keys_Pilot/fr

redo_job*: No such file or directory
Killed by signal 1.
